In [1]:
# import world
import os
import time
from retrievalHelper.utils import *
from retrievalHelper.u4Res import *
from retrievalHelper.u4KNN import *
from retrievalHelper.u4train import *
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn.init import xavier_normal_, constant_, xavier_uniform_
import torch.optim as optim
GPU = torch.cuda.is_available()
device = torch.device('cuda' if GPU else "cpu")


In [2]:
config = {}
config['city'] = "Edinburgh"
config['checkKeyword'] = False
config['showCheck']= False
config['quantity'] = 20
config['seed'] = 1001
config['edgeType']  = "IUF"
config['logResult'] = "log"
config['export2LLMs'] = True
config['RetModel'] = "CBR"
config['numKW4FT'] = 20

modelConfig = {}
modelConfig['hidden_dim'] = 32
modelConfig['learning_rate'] = 1e-3
modelConfig['num_epochs'] = 1

In [3]:
print("Running retrieval... ")
print("Starting time: ", time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))
print("INFO: ", config)

city = config['city']
print("Loading training keyword")
trainDat, testDat = data_reviewLoader(city)

train_users, train_users2kw = extract_users(trainDat['np2users'])
test_users, test_users2kw = extract_users(testDat['np2users'])

# extract user2rest for label
groundtruth = load_groundTruth(f'./data/reviews/{city}.csv')

keywordScore, keywordFrequence = load_kwScore(city, config["edgeType"])


restGraph = retaurantReviewG([trainDat, keywordScore, keywordFrequence, \
                                config["quantity"], config["edgeType"], groundtruth])

KNN = neighbor4kw(f'{city}_kwSenEB_pad', testDat,  restGraph)
rest_Label = getRestLB(trainDat['np2rests'])

sourceFile = open(config["logResult"], 'a')

Running retrieval... 
Starting time:  2024-05-16 02:49:18
INFO:  {'city': 'Edinburgh', 'checkKeyword': False, 'showCheck': False, 'quantity': 20, 'seed': 1001, 'edgeType': 'IUF', 'logResult': 'log', 'export2LLMs': True, 'RetModel': 'CBR', 'numKW4FT': 20}
Loading training keyword
info from extracted file
Number of keyword: 75152
info from extracted file
Number of keyword: 20495
number of review:  12753


In [4]:
mp, mr, mf = 0, 0, 0
dictionary = {}


In [5]:
userFT, userFT_test, rest_feature = KNN.loadFT(config["numKW4FT"], rest_Label, config["city"])
label_train, label_test = label_ftColab(train_users, test_users, groundtruth, restGraph.numRest, rest_Label)
dim_users, dim_items = 384, 384
learning_rate = modelConfig["learning_rate"]
hidden_dim = modelConfig['hidden_dim']
num_epochs = modelConfig['num_epochs']

trainLB = np.asarray(label_train)
testLB = np.asarray(label_test)
print("feature shape (train / test):")
print(userFT.shape, userFT_test.shape)


100%|█████████████████████████████████████████████████████████████████████████| 1187/1187 [00:00<00:00, 1379.05it/s]
/opt/anaconda3/envs/graph/lib/python3.9/site-packages/threadpoolctl.py:1223: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
100%|██████████████████████████████████████████████████████████████████████████| 223/223 [00:00<00:00, 12756.82it/s]

feature shape (train / test):
(1187, 20, 384) (223, 20, 384)


In [6]:
train_dataset = DataCF(userFT, trainLB)
test_dataset = DataCF(userFT_test, testLB)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
rest_feature = torch.from_numpy(rest_feature).type(torch.FloatTensor)
rest_feature = rest_feature.to(device)


In [7]:
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn.init import xavier_normal_, constant_, xavier_uniform_
import torch.optim as optim
import torch


class MatrixFactorization(nn.Module):
    def __init__(self, dim_users, dim_items, embedding_dim):
        super(MatrixFactorization, self).__init__()
        self.user_embeddings = nn.Linear(dim_users, embedding_dim)
        self.uAP = AttentionPooling(embedding_dim , embedding_dim // 4)
        self.item_embeddings = nn.Linear(dim_items, embedding_dim)
        self.iAP = AttentionPooling(embedding_dim , embedding_dim // 4)
        self.dropout = nn.Dropout(0.2)
        self.activate = nn.Sigmoid()
        # initial weight
        self.apply(xavier_normal_initialization)

    def forward(self, user_ft, item_ft):
        numBatch = len(user_ft)
        numRest = len(item_ft)
        user_embeddings = self.user_embeddings(user_ft)
        # user_embeddings = self.r1(user_embeddings)
        user_embeddings = self.dropout(user_embeddings)
        user_embeddings, _ = self.uAP(user_embeddings)
        item_embeddings = self.item_embeddings(item_ft)
        item_embeddings = self.dropout(item_embeddings)
        # item_embeddings = self.r2(item_embeddings)
        item_embeddings, _ = self.iAP(item_embeddings)
        item_embeddings = torch.permute(item_embeddings, (1, 0))
        pred = self.activate(torch.matmul(user_embeddings, item_embeddings))
        # using this to return 
        return pred   


    def prediction(self, user_ft, item_ft):
        return self.forward(user_ft, item_ft)

class AttentionPooling(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(AttentionPooling, self).__init__()

        # Linear layers for attention scoring
        self.V = nn.Linear(input_size, hidden_size)
        self.size = input_size
        self.w = nn.Linear(hidden_size, 1)
        self.tanh = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_features):
        # Calculate attention scores
        scores = self.tanh(self.V(input_features)) 
        scores = self.w(scores)
        
        # Apply softmax to get attention weights
        weights = self.softmax(scores)

        # Apply attention weights to input features
        pooled_features = torch.sum(weights * input_features, dim=1)

        return pooled_features, weights

def xavier_normal_initialization(module):
    r""" using `xavier_normal_`_ in PyTorch to initialize the parameters in
    nn.Embedding and nn.Linear layers. For bias in nn.Linear layers,
    using constant 0 to initialize.
    .. _`xavier_normal_`:
        https://pytorch.org/docs/stable/nn.init.html?highlight=xavier_normal_#torch.nn.init.xavier_normal_
    Examples:
        >>> self.apply(xavier_normal_initialization)
    """
    if isinstance(module, nn.Linear):
        xavier_normal_(module.weight.data)
        if module.bias is not None:
            constant_(module.bias.data, 0)  
def extractResult(lResults):
    p = [x[0] for x in lResults]
    r = [x[1] for x in lResults]
    f = [x[2] for x in lResults]
    return p, r, f

In [10]:
model = MatrixFactorization(dim_users, dim_items, hidden_dim).to(device)
print(model)


MatrixFactorization(
  (user_embeddings): Linear(in_features=384, out_features=32, bias=True)
  (uAP): AttentionPooling(
    (V): Linear(in_features=32, out_features=8, bias=True)
    (w): Linear(in_features=8, out_features=1, bias=True)
    (tanh): Sigmoid()
    (softmax): Softmax(dim=1)
  )
  (item_embeddings): Linear(in_features=384, out_features=32, bias=True)
  (iAP): AttentionPooling(
    (V): Linear(in_features=32, out_features=8, bias=True)
    (w): Linear(in_features=8, out_features=1, bias=True)
    (tanh): Sigmoid()
    (softmax): Softmax(dim=1)
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (activate): Sigmoid()
)


In [12]:
optimizer = optim.AdamW(model.parameters(), lr=0.03, weight_decay= 1e-4)
criterion = nn.BCELoss()
# Training loop
for epoch in range(500):
    total_loss = 0.0
    model.train()
    for batch_idx, batchDat in enumerate(train_loader):
        optimizer.zero_grad()
        data, label = batchDat
        userDat = data.to(device)
        label = label.to(device)
        restDat = rest_feature
        prediction = model(userDat, restDat)
        loss = criterion(prediction , label)
    loss.backward()
    total_loss += loss.item() 
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss}')
    if (epoch+1) % 10 == 0:
        
	        lResults = evaluateModel(model, test_loader, rest_feature, groundtruth, test_users, config['quantity'], rest_Label)

	        p, r, f = extractResult(lResults)
	        if mean(r) > mean(mr):
	            mp, mr, mf = p, r, f
	        print(f'Epoch [{epoch+1}/{num_epochs}], prec: {mean(p)}, rec: {mean(r)}, f1: {mean(f)}')
	    

Epoch [1/1], Loss: 0.05546272173523903
Epoch [2/1], Loss: 0.20132334530353546
Epoch [3/1], Loss: 0.043773941695690155
Epoch [4/1], Loss: 1.9901691675186157
Epoch [5/1], Loss: 0.04966359585523605
Epoch [6/1], Loss: 0.11673018336296082
Epoch [7/1], Loss: 0.18982993066310883
Epoch [8/1], Loss: 0.1188897043466568
Epoch [9/1], Loss: 0.19427472352981567
Epoch [10/1], Loss: 0.14181426167488098
Epoch [10/1], prec: 0.0008968609865470852, rec: 0.0009777889267843522, f1: 0.0008223368786411005
Epoch [11/1], Loss: 0.370719313621521
Epoch [12/1], Loss: 0.319380521774292
Epoch [13/1], Loss: 0.1858205646276474
Epoch [14/1], Loss: 0.27345970273017883
Epoch [15/1], Loss: 0.25725066661834717
Epoch [16/1], Loss: 0.49446627497673035
Epoch [17/1], Loss: 0.46936720609664917
Epoch [18/1], Loss: 0.3756892681121826
Epoch [19/1], Loss: 0.570432186126709
Epoch [20/1], Loss: 0.4567241072654724
Epoch [20/1], prec: 0.0011210762331838565, rec: 0.001181620969181417, f1: 0.001035875208771359
Epoch [21/1], Loss: 0.43314

KeyboardInterrupt: 